### Loading data from cluster

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(2024)

from skimage.transform import resize
import pywt
import os
import scipy.io.wavfile as wavfile
from scipy.signal import correlate, hilbert, butter, firwin, filtfilt, lfilter 
import scipy.io
from scipy.signal import resample_poly
from scipy.spatial.distance import cdist
from IPython.display import display, Audio

import warnings
warnings.filterwarnings('ignore')

In [ ]:
file_path='/labs/cliffordlab/fetal/leipzig/Version2/'
Annot_path_1='/labs/cliffordlab/fetal/leipzig/Annotations/ManualAnnotations/Beat_annotationAnno1'
Annot_path_2='/labs/cliffordlab/fetal/leipzig/Annotations/ManualAnnotations/Beat_annotationAnno2'

file_names=['CB300482IIIV2','MZ290383V2','RM040883IIV2','SB280780_1V2', 'AK101197V2', 'BA120381V2','KF221288V2','SZ290877V2']

signals = [np.squeeze(scipy.io.loadmat(file_path+file)['doppler']) for file in file_names]
ECGs=[np.squeeze(scipy.io.loadmat(file_path+file)['fECG']) for file in file_names]

fECG_1 = scipy.io.loadmat('./AK101197_fECG.mat')['all']
fECG_6 = scipy.io.loadmat('./BA120381_fECG.mat')['all']
fECG_7 = scipy.io.loadmat('./KF221288_fECG.mat')['all']
fECG_8 = scipy.io.loadmat('./SZ290877_fECG.mat')['all']

ECGs_2 = ECGs[:-4]
ECGs_2.append(fECG_1)
ECGs_2.append(fECG_6)
ECGs_2.append(fECG_7)
ECGs_2.append(fECG_8)
ECG_sigs = ECGs_2

In [ ]:
SQI_1 = []
SQI_2 = []
ECG_1=[]
ECG_2=[]
ECG_SQIC_1=[]
ECG_SQIC_2=[]
ECG_SQI_1=[]
ECG_SQI_2=[]

file_names_old=['CB300482IIIV2','MZ290383V2','RM040883IIV2','SB280780_1V2', 'AK101197V2']

for file in file_names_old:
    SQI_path_1 = Annot_path_1 + '/' + file + '_Doppler_SQI.txt'
    SQI_path_2 = Annot_path_2 + '/' + file + '_Doppler_SQI.txt'
    
    SQI_data_1 = np.loadtxt(SQI_path_1, delimiter=',')
    SQI_1.append(SQI_data_1)
    
    SQI_data_2 = np.loadtxt(SQI_path_2, delimiter=',')
    SQI_2.append(SQI_data_2)
    
    ECG_SQIC_1=[]
    ECG_SQIC_2=[]
    for i in range(1,8):
        ECG_path_1 = Annot_path_1 + '/' + file + f'_{i}_SQI.txt'
        ECG_path_2 = Annot_path_2 + '/' + file + f'_{i}_SQI.txt'
        
        ECG_SQI1 = np.loadtxt(ECG_path_1, delimiter=',')
        ECG_SQIC_1.append(ECG_SQI1)
        ECG_SQI2 = np.loadtxt(ECG_path_2, delimiter=',')
        ECG_SQIC_2.append(ECG_SQI2)
        
    ECG_SQI_1.append(ECG_SQIC_1)
    ECG_SQI_2.append(ECG_SQIC_2)
    
file_names_new=['BA120381V2','KF221288V2','SZ290877V2']

for file in file_names_new:
    SQI_path_1 = './Additional_data/SQIs/' + file + '_Doppler_SQI.txt'
    SQI_path_2 = './Additional_data/SQIs/' + file + '_Doppler_SQI.txt'
    
    SQI_data_1 = np.loadtxt(SQI_path_1, delimiter=',')
    SQI_1.append(SQI_data_1)
    
    SQI_data_2 = np.loadtxt(SQI_path_2, delimiter=',')
    SQI_2.append(SQI_data_2)
    
    ECG_SQIC_1=[]
    ECG_SQIC_2=[]
    for i in range(1,8):
        ECG_path_1 = './Additional_data/SQIs/' + file + f'_{i}_SQI.txt'
        ECG_path_2 = './Additional_data/SQIs/' + file + f'_{i}_SQI.txt'
        
        ECG_SQI1 = np.loadtxt(ECG_path_1, delimiter=',')
        ECG_SQIC_1.append(ECG_SQI1)
        ECG_SQI2 = np.loadtxt(ECG_path_2, delimiter=',')
        ECG_SQIC_2.append(ECG_SQI2)
        
    ECG_SQI_1.append(ECG_SQIC_1)
    ECG_SQI_2.append(ECG_SQIC_2)

In [ ]:
ECG_1=[]
ECG_2=[]

for file in file_names:
    if file == 'SZ290877V2':
        ECG_1.append(np.loadtxt('./Additional_data/SQIs/SZ290877V2_2.txt', delimiter=','))
        ECG_2.append(np.loadtxt('./Additional_data/SQIs/SZ290877V2_2.txt', delimiter=','))
    elif file == 'KF221288V2':
        ECG_1.append(np.loadtxt('./Additional_data/SQIs/KF221288V2_2.txt', delimiter=','))
        ECG_2.append(np.loadtxt('./Additional_data/SQIs/KF221288V2_2.txt', delimiter=','))
    elif file == 'BA120381V2':
        ECG_path_1 = Annot_path_1 + '/' + file + '_2.txt'
        ECG_data_1 = np.loadtxt(ECG_path_1, delimiter=',')
        ECG_1.append(ECG_data_1[:,0])
        ECG_2.append(ECG_data_1[:,0])

    else:
        ECG_path_1 = Annot_path_1 + '/' + file + '_2.txt'
        ECG_path_2 = Annot_path_2 + '/' + file + '_2.txt'

        ECG_data_1 = np.loadtxt(ECG_path_1, delimiter=',')
        ECG_1.append(ECG_data_1[:,0])
        
        ECG_data_2 = np.loadtxt(ECG_path_2, delimiter=',')
        ECG_2.append(ECG_data_2[:,0])

In [ ]:
def find_closest_index(lst, number):
    differences = [abs(x - number) for x in lst]
    closest_index = differences.index(min(differences))

    return closest_index

def modify_array(arr, limit):
    length = arr.shape[0]
    if length > limit:
        return arr[:limit]
    elif length < limit:
        padding_length = limit - length
        return np.pad(arr, (0, padding_length), 'constant', constant_values=(np.mean(arr),))
    else:
        return arr

def normalize_one(array):
    ecg_min = np.min(array)
    ecg_max = np.max(array)
    
    normalized_ecg = 2 * (array - ecg_min) / (ecg_max - ecg_min) - 1
    
    return normalized_ecg
    
def normalize(array):
    return (array - array.min())/(array.max() - array.min()) 

def normalize_mean_std(array):
    mean = np.mean(array)
    std = np.std(array)
    normalized_array = (array - mean) / std
    return normalized_array


def convert_to_float(byte_string):
    string = byte_string.decode('utf-8')  # Decode byte string to a regular string
    number_str = string.split(',')[0]  # Split at the comma and take the first part
    return float(number_str)  # Convert to float

def butter_bandpass(lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def DUS_filtering(DUS, fs):  
    #25-600Hz 4th order Butterworth band pass
    lowcut = 25.0
    highcut = 600.0
    DUS_f = butter_bandpass_filter(DUS, lowcut, highcut, fs, order=2)
    return DUS_f

def signal_resample(signal, original_fs, target_fs):
    gcd = np.gcd(original_fs, target_fs)
    up = target_fs // gcd
    down = original_fs // gcd
    resampled_signal=resample_poly(signal, up, down)
    return resampled_signal


def ECG_filtering(ecg_signal, sample_rate):
    pass_band = [3, 45]  # Pass band frequencies (Hz)
    
    # Nyquist frequency
    nyquist_rate = sample_rate / 2.0
    
    # Calculate the filter coefficients
    numtaps = 101  # Number of taps in the FIR filter
    fir_coefficients = firwin(numtaps, [pass_band[0] / nyquist_rate, pass_band[1] / nyquist_rate], pass_zero=False)
    
    # Apply the filter
    filtered_signal = filtfilt(fir_coefficients, 1, ecg_signal)
    
    return filtered_signal



def findClosersDuplicates(beats, threshold):
    """
    Remove duplicates from beats array based on a threshold.
    """
    unique_beats = []
    prev_beat = -np.inf
    for beat in sorted(beats):
        if beat - prev_beat > threshold:
            unique_beats.append(beat)
            prev_beat = beat
    return np.array(unique_beats)

def groupingBeats_leipzig(beats1, beats2, fs):
    beats1 = findClosersDuplicates(beats1, 0.24 * fs)
    beats2 = findClosersDuplicates(beats2, 0.24 * fs)

    D = cdist(beats1[:, np.newaxis], beats2[:, np.newaxis]) #distance between a beat from beats1 and a beat from beats2

    idxMinBeats1 = np.argmin(D, axis=1)
    idxMinBeats2 = np.argmin(D, axis=0)

    beatSet = []
    
    if len(np.unique(idxMinBeats1)) == len(beats1):
        for i in range(len(beats1)):
            beatSet.append(np.mean([beats1[i], beats2[idxMinBeats1[i]]]))
        
        if len(beats1) != len(beats2):
            missingBeats2 = np.setdiff1d(range(len(beats2)), idxMinBeats1)
            for i in missingBeats2:
                beatSet.append(beats2[i])
    else:
        uniqueBeats2 = np.unique(idxMinBeats1)
        
        for i in uniqueBeats2:
            idxBeat1 = np.where(idxMinBeats1 == i)[0]
            
            if len(idxBeat1) == 1:
                beatSet.append(np.mean([beats1[idxBeat1[0]], beats2[i]]))
            else:
                closerBeat1FromBeat2 = idxMinBeats2[i]
                beatSet.append(np.mean([beats1[closerBeat1FromBeat2], beats2[i]]))
                extras = beats1[np.setdiff1d(idxBeat1, closerBeat1FromBeat2)]
                beatSet.extend(extras)

    return sorted(beatSet)

In [ ]:
ECG_ranking_indices = scipy.io.loadmat('./VotingRanking_allch_DSQI.mat')['indices']
ECG_ranking_indices_all = []
def extract_numbers(data):
    # Initialize an empty list to store the results
    result = []
    # Loop through each row
    for row in data:
        # Loop through each column in the row
        inner_list = []
        for col in row:
            # Extract the innermost number and append to the inner list
            inner_list.append(col[0][0])
        # Append the inner list to the result list
        result.append(inner_list)
    return np.array(result)
    
indices_data_1 = extract_numbers(ECG_ranking_indices[0][0])
indices_data_2 = extract_numbers(ECG_ranking_indices[1][0])
indices_data_3 = extract_numbers(ECG_ranking_indices[2][0])
indices_data_4 = extract_numbers(ECG_ranking_indices[3][0])
indices_data_5 = extract_numbers(ECG_ranking_indices[4][0])
indices_data_6 = extract_numbers(ECG_ranking_indices[5][0])
indices_data_7 = extract_numbers(ECG_ranking_indices[6][0])
indices_data_8 = extract_numbers(ECG_ranking_indices[7][0])

ECG_ranking_indices_all.append(indices_data_1[:,-1])
ECG_ranking_indices_all.append(indices_data_2[:,-1])
ECG_ranking_indices_all.append(indices_data_3[:,-1])
ECG_ranking_indices_all.append(indices_data_4[:,-1])
ECG_ranking_indices_all.append(indices_data_5[:,-1])
ECG_ranking_indices_all.append(indices_data_6[:,-1])
ECG_ranking_indices_all.append(indices_data_7[:,-1])
ECG_ranking_indices_all.append(indices_data_8[:,-1])

#### Doppler-fECG heartbeat extraction

In [ ]:
fs_ECG=1000
target_fs_Doppler=2000
target_fs_ECG=250

win_len=3.75

DUS_list_all_1 = []
ECG_list_all_1 = []
DUS_list_all_2 = []
ECG_list_all_2 = []
DUS_list_all_3 = []
ECG_list_all_3 = []
DUS_list_all_4 = []
ECG_list_all_4 = []
DUS_list_all_5 = []
ECG_list_all_5 = []
DUS_list_all_6 = []
ECG_list_all_6 = []
DUS_list_all_7 = []
ECG_list_all_7 = []
DUS_list_all_8 = []
ECG_list_all_8 = []

DUS_list_all_train_1 = []
ECG_list_all_train_1 = []
DUS_list_all_train_2 = []
ECG_list_all_train_2 = []
DUS_list_all_train_3 = []
ECG_list_all_train_3 = []
DUS_list_all_train_4 = []
ECG_list_all_train_4 = []
DUS_list_all_train_5 = []
ECG_list_all_train_5 = []
DUS_list_all_train_6 = []
ECG_list_all_train_6 = []
DUS_list_all_train_7 = []
ECG_list_all_train_7 = []
DUS_list_all_train_8 = []
ECG_list_all_train_8 = []
    
tensor_all_list_all = []
label_list_all = []
beatset_list_all= []
FHR_list_all= []
DUS_list_all = []
ECG_list_all = []

adj = [-17,-13,-29,-11,-14,0,0,2]

t_Doppler = np.linspace(0, 3.75, 7500)
t_ECG = np.linspace(0, 3.75, 938)
g_segments = 0

for signal_num, signal_name in enumerate(file_names):
    print(signal_name)
    
    if file_names[signal_num]=='MZ290383V2':
        fs_Doppler=10000
    else:
        fs_Doppler=20000
        
    SQI_len=min(len(SQI_1[signal_num]),len(SQI_2[signal_num]))
    
    for page in range(SQI_len):
        
        if SQI_1[signal_num][page]==1 and SQI_2[signal_num][page]==1:

            start_time=page*win_len
            end_time=(page+1)*win_len

            start_Index_ECG = start_time*fs_ECG
            end_Index_ECG = end_time*fs_ECG

            start_Index_Doppler = start_time*fs_Doppler
            end_Index_Doppler = end_time*fs_Doppler

            ECG_page_anno1 = ECG_1[signal_num][(ECG_1[signal_num] >= start_Index_ECG) & (ECG_1[signal_num] <= end_Index_ECG)]
            ECG_page_anno2 = ECG_2[signal_num][(ECG_2[signal_num] >= start_Index_ECG) & (ECG_2[signal_num] <= end_Index_ECG)]


            beatSet = groupingBeats_leipzig(ECG_page_anno1, ECG_page_anno2,fs_ECG)
            beatSet_time=np.array(beatSet)/fs_ECG-start_time


            diffFHR = []
            for bIdx in range(1, len(beatSet)):
                if beatSet[bIdx] - beatSet[bIdx - 1] <= fs_ECG * (2 / 3): #Greater than 90 bpm
                    diffFHR.append(beatSet[bIdx] - beatSet[bIdx - 1])
                    
            #FHR_all=60 / (np.array(diffFHR) / fs_ECG)
            #FHR = 60 / np.median(np.array(diffFHR) / fs_ECG)

            ECG_SQI_CH_1 = []
            ECG_SQI_CH_2 = []
            
            for i in range(7): 
                ECG_SQI_CH_1.append(ECG_SQI_1[signal_num][i][page])
                ECG_SQI_CH_2.append(ECG_SQI_2[signal_num][i][page])
                
            one_sqi_channels = np.where((np.array(ECG_SQI_CH_1) == 1) & (np.array(ECG_SQI_CH_2) == 1))[0]

            if one_sqi_channels.size > 1:
                ranking = ECG_ranking_indices_all[signal_num][page*7:(page+1)*7]
                set_intersection = set(one_sqi_channels).intersection(ranking)
                sqi = [x for x in one_sqi_channels if x in set_intersection][0]
            elif one_sqi_channels.size == 1:
                sqi = one_sqi_channels[0]

            sqi_sum = [x + y for x, y in zip(ECG_SQI_CH_1, ECG_SQI_CH_2)]
            if all(value > 5 for value in sqi_sum) or ECG_SQI_CH_1 == 5 or ECG_SQI_CH_2 == 5:
                continue
            
            else:
                sqi_sum = [x + y for x, y in zip(ECG_SQI_CH_1, ECG_SQI_CH_2)]
                min_sqi_sum = np.min(sqi_sum)
                candidates = np.where(sqi_sum == min_sqi_sum)[0]
                if len(candidates) > 1:
                    cd_inx = candidates.tolist()[0]
                    min_sqi_1 = np.min(ECG_SQI_CH_1[cd_inx])
                    min_sqi_2 = np.min(ECG_SQI_CH_2[cd_inx])
            
                    final_candidates = [c for c in candidates if ECG_SQI_CH_1[c] == min_sqi_1 or ECG_SQI_CH_2[c] == min_sqi_2]
            
                    if len(final_candidates) > 1:
                        ranking = ECG_ranking_indices_all[signal_num][page*7:(page+1)*7]
                        set_intersection = set(final_candidates).intersection(ranking)
                        sqi = [c for c in final_candidates if c in set_intersection][0]
                    else:
                        sqi = int(final_candidates[0])
                else:
                    sqi = int(candidates[0])

            g_segments += 1
            ECG=ECG_sigs[signal_num][sqi][int(start_Index_ECG):int(end_Index_ECG)]
            ECG=signal_resample(ECG, fs_ECG, target_fs_ECG)
            ECG=ECG_filtering(ECG, target_fs_ECG)
            ECG=np.roll(normalize_one(ECG), adj[signal_num])

            windowed_rec_original=signals[signal_num][int(start_Index_Doppler):int(end_Index_Doppler)]

            windowed_rec_resampled=signal_resample(windowed_rec_original, fs_Doppler, target_fs_Doppler)
            windowed_rec=DUS_filtering(windowed_rec_resampled, target_fs_Doppler)
            windowed_rec=normalize_one(windowed_rec)

            for i in range(len(beatSet_time)):
                if (i == len(beatSet_time) - 2):
                    break
                
                i_d = find_closest_index(t_Doppler, beatSet_time[i])
                i_d_1 = find_closest_index(t_Doppler, beatSet_time[i+1])
                i_d_2 = find_closest_index(t_Doppler, beatSet_time[i+2])
                
                i_t = find_closest_index(t_ECG, beatSet_time[i])
                i_t_1 = find_closest_index(t_ECG, beatSet_time[i+1])
                i_t_2 = find_closest_index(t_ECG, beatSet_time[i+2])


                windowed_rec_beat = windowed_rec[round((i_d_1+i_d)/2):round((i_d_2+i_d_1)/2)]
                windowed_rec_beat = normalize_one(windowed_rec_beat)
                windowed_rec_beat = modify_array(windowed_rec_beat,800)
                
                ECG_beat = ECG[round((i_t_1+i_t)/2):round((i_t_2+i_t_1)/2)]
                ECG_beat=normalize_one(ECG_beat)
                ECG_beat = modify_array(ECG_beat,100)


                if file_names[signal_num]=='AK101197V2':
                    DUS_list_all_1.append(windowed_rec_beat)
                    ECG_list_all_1.append(ECG_beat)
                               
                if file_names[signal_num]=='CB300482IIIV2':
                    DUS_list_all_2.append(windowed_rec_beat)
                    ECG_list_all_2.append(ECG_beat)
                                
                if file_names[signal_num]=='MZ290383V2':
                    DUS_list_all_3.append(windowed_rec_beat)
                    ECG_list_all_3.append(ECG_beat)                    
                               
                if file_names[signal_num]=='RM040883IIV2':
                    DUS_list_all_4.append(windowed_rec_beat)
                    ECG_list_all_4.append(ECG_beat)                   
                                
                if file_names[signal_num]=='SB280780_1V2':
                    DUS_list_all_5.append(windowed_rec_beat)
                    ECG_list_all_5.append(ECG_beat)    

                if file_names[signal_num]=='BA120381V2':
                    DUS_list_all_6.append(windowed_rec_beat)
                    ECG_list_all_6.append(ECG_beat)
                               
                if file_names[signal_num]=='KF221288V2':
                    DUS_list_all_7.append(windowed_rec_beat)
                    ECG_list_all_7.append(ECG_beat)
                                
                if file_names[signal_num]=='SZ290877V2':
                    DUS_list_all_8.append(windowed_rec_beat)
                    ECG_list_all_8.append(ECG_beat)                    
                               

DUS_list_all_1=np.array(DUS_list_all_1, dtype=object)
ECG_list_all_1=np.array(ECG_list_all_1, dtype=object)
DUS_list_all_2=np.array(DUS_list_all_2, dtype=object)
ECG_list_all_2=np.array(ECG_list_all_2, dtype=object)
DUS_list_all_3=np.array(DUS_list_all_3, dtype=object)
ECG_list_all_3=np.array(ECG_list_all_3, dtype=object)
DUS_list_all_4=np.array(DUS_list_all_4, dtype=object)
ECG_list_all_4=np.array(ECG_list_all_4, dtype=object)
DUS_list_all_5=np.array(DUS_list_all_5, dtype=object)
ECG_list_all_5=np.array(ECG_list_all_5, dtype=object)
DUS_list_all_6=np.array(DUS_list_all_6, dtype=object)
ECG_list_all_6=np.array(ECG_list_all_6, dtype=object)
DUS_list_all_7=np.array(DUS_list_all_7, dtype=object)
ECG_list_all_7=np.array(ECG_list_all_7, dtype=object)
DUS_list_all_8=np.array(DUS_list_all_8, dtype=object)
ECG_list_all_8=np.array(ECG_list_all_8, dtype=object)


np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_1.npz', DUS_list_all=DUS_list_all_1, ECG_list_all=ECG_list_all_1)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_2.npz', DUS_list_all=DUS_list_all_2, ECG_list_all=ECG_list_all_2)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_3.npz', DUS_list_all=DUS_list_all_3, ECG_list_all=ECG_list_all_3)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_4.npz', DUS_list_all=DUS_list_all_4, ECG_list_all=ECG_list_all_4)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_5.npz', DUS_list_all=DUS_list_all_5, ECG_list_all=ECG_list_all_5)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_6.npz', DUS_list_all=DUS_list_all_6, ECG_list_all=ECG_list_all_6)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_7.npz', DUS_list_all=DUS_list_all_7, ECG_list_all=ECG_list_all_7)
np.savez('./Train_addition/Leipzing_heartbeat_DUS_FECG_8.npz', DUS_list_all=DUS_list_all_8, ECG_list_all=ECG_list_all_8)

### ML model SQI prediction

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from fastdtw import fastdtw
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, f1_score, RocCurveDisplay
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score
from scipy.stats import pearsonr
from scipy.signal import butter, filtfilt, hilbert, welch, lfilter, correlate
from scipy.stats import weightedtau
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from itertools import cycle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import seaborn as sns
from scipy.spatial.distance import euclidean
from scipy.stats import skew, kurtosis
from catboost import CatBoostClassifier
import nolds
from sklearn.preprocessing import StandardScaler
import pywt

In [ ]:
ECG_6 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/ECG_6_all_ch_ASQI.mat')['data']
ECG_7 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/ECG_7_all_ch_ASQI.mat')['data']
ECG_8 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/ECG_8_all_ch_ASQI.mat')['data']

Doppler_6 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/Doppler_6_ASQI.mat')['data'].T
Doppler_7 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/Doppler_7_ASQI.mat')['data'].T
Doppler_8 = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/Doppler_8_ASQI.mat')['data'].T

indices_add = scipy.io.loadmat('/Users/alirezarafiei/Desktop/sqi/ECG_indices_additional_data_ASQI.mat')['indices']

dtw_dist_lag_6 = np.load('/Users/alirezarafiei/Desktop/sqi/dtw_dist_lag_6.npy')
dtw_dist_lag_7 = np.load('/Users/alirezarafiei/Desktop/sqi/dtw_dist_lag_7.npy')
dtw_dist_lag_8 = np.load('/Users/alirezarafiei/Desktop/sqi/dtw_dist_lag_8.npy')

correlations_lag_6 = np.load('/Users/alirezarafiei/Desktop/sqi/correlations_lag_6.npy')
correlations_lag_7 = np.load('/Users/alirezarafiei/Desktop/sqi/correlations_lag_7.npy')
correlations_lag_8 = np.load('/Users/alirezarafiei/Desktop/sqi/correlations_lag_8.npy')

In [ ]:
def extract_numbers(data):
    # Initialize an empty list to store the results
    result = []
    # Loop through each row
    for row in data:
        # Loop through each column in the row
        inner_list = []
        for col in row:
            # Extract the innermost number and append to the inner list
            inner_list.append(col[0][0])
        # Append the inner list to the result list
        result.append(inner_list)
    return np.array(result)
Feature_indices_6 = extract_numbers(indices_add[0][0])
Feature_indices_7 = extract_numbers(indices_add[1][0])
Feature_indices_8 = extract_numbers(indices_add[2][0])

indices_indices_all = np.concatenate([Feature_indices_6,Feature_indices_7,Feature_indices_8])

In [ ]:
def calculate_envelope(signal):
    analytic_signal = hilbert(signal)
    envelope = np.abs(analytic_signal)
    return envelope

def homomorphic_envelope_processed(input_signal, fs=20000, lpf_frequency=8):
    B_low, A_low = butter(1, 2 * lpf_frequency / fs, 'low')
    analytic_signal = hilbert(input_signal)
    amplitude_envelope = np.abs(analytic_signal)
    log_amplitude_envelope = np.log(amplitude_envelope)
    filtered_log_envelope = filtfilt(B_low, A_low, log_amplitude_envelope)
    homomorphic_envelope = np.exp(filtered_log_envelope)
    
    # Remove spurious spikes in first sample
    if len(homomorphic_envelope) > 1:
        homomorphic_envelope[0] = homomorphic_envelope[1]

    # Min-max normalization to range [0, 1]
    if homomorphic_envelope.ndim == 2:
        normalized_homomorphic_envelope = np.zeros_like(homomorphic_envelope)
        for i in range(homomorphic_envelope.shape[0]):
            min_val = np.min(homomorphic_envelope[i, :])
            max_val = np.max(homomorphic_envelope[i, :])
            normalized_homomorphic_envelope[i, :] = (homomorphic_envelope[i, :] - min_val) / (max_val - min_val)
    else:
        min_val = np.min(homomorphic_envelope)
        max_val = np.max(homomorphic_envelope)
        normalized_homomorphic_envelope = (homomorphic_envelope - min_val) / (max_val - min_val)
    
    return normalized_homomorphic_envelope

def normalize_zero_one(array):
    ecg_min = np.min(array)
    ecg_max = np.max(array)
    
    normalized_ecg = 2 * (array - ecg_min) / (ecg_max - ecg_min)

    if array.ndim == 2:
        normalized_array = np.zeros_like(array)
        for i in range(array.shape[0]):
            min_val = np.min(array[i, :])
            max_val = np.max(array[i, :])
            normalized_array[i, :] = (array[i, :] - min_val) / (max_val - min_val)
    else:
        min_val = np.min(array)
        max_val = np.max(array)
        normalized_array = (array - min_val) / (max_val - min_val)
    
    return normalized_array
def butter_bandpass(data, lowcut, highcut, fs=1000, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def pan_tompkins_envelope_processed(ecg_signals, lowcut=8, highcut=80, fs=1000):
    envelopes = []

    for ecg_signal in ecg_signals:
        # Filter ECG signal between 8 Hz and 80 Hz
        ecg_filtered = butter_bandpass(ecg_signal, lowcut, highcut, fs)

        # Differentiation
        diff = np.diff(ecg_filtered, prepend=ecg_filtered[0])

        # Squaring
        squared = diff**2

        # Moving Window Integration
        window_length = int(0.05 * fs)  
        window = np.ones(window_length) / window_length
        integrated = np.convolve(squared, window, mode='same')

        # Extracting the envelope using the Hilbert transform
        envelope = np.abs(hilbert(integrated))
        envelopes.append(envelope)

    return np.array(envelopes)

In [ ]:
Doppler_6_7 = np.repeat(Doppler_6, 7, axis=0)
Doppler_7_7 = np.repeat(Doppler_7, 7, axis=0)
Doppler_8_7 = np.repeat(Doppler_8, 7, axis=0)

envelope_D_6_7 = homomorphic_envelope_processed(Doppler_6_7)
envelope_D_7_7 = homomorphic_envelope_processed(Doppler_7_7)
envelope_D_8_7 = homomorphic_envelope_processed(Doppler_8_7, fs=10000)

envelope_D_6_7_r = np.mean(envelope_D_6_7.reshape(len(envelope_D_6_7), 3750, -1), axis=2)
envelope_D_7_7_r = np.mean(envelope_D_7_7.reshape(len(envelope_D_7_7), 3750, -1), axis=2)
envelope_D_8_7_r = np.mean(envelope_D_8_7.reshape(len(envelope_D_8_7), 3750, -1), axis=2)


ECG_6_rearrange = np.reshape(ECG_6,(len(ECG_6)*7,3750))
ECG_7_rearrange = np.reshape(ECG_7,(len(ECG_7)*7,3750))
ECG_8_rearrange = np.reshape(ECG_8,(len(ECG_8)*7,3750))

ECG_6_normalized = normalize_zero_one(ECG_6_rearrange)
ECG_7_normalized = normalize_zero_one(ECG_7_rearrange)
ECG_8_normalized = normalize_zero_one(ECG_8_rearrange)

envelope_E_6 = homomorphic_envelope_processed(ECG_6_rearrange,fs=1000)
envelope_E_7 = homomorphic_envelope_processed(ECG_7_rearrange,fs=1000)
envelope_E_8 = homomorphic_envelope_processed(ECG_8_rearrange,fs=1000)


pan_envelope_E_6 = pan_tompkins_envelope_processed(ECG_6_rearrange,fs=1000)
pan_envelope_E_7 = pan_tompkins_envelope_processed(ECG_7_rearrange,fs=1000)
pan_envelope_E_8 = pan_tompkins_envelope_processed(ECG_8_rearrange,fs=1000)


In [ ]:
def calculate_entropy(signal):
    histogram, _ = np.histogram(signal, bins=64, density=True)
    histogram = histogram[histogram > 0]  # Remove zero entries
    entropy = -np.sum(histogram * np.log2(histogram))
    return entropy

def calculate_psd(signal, fs=1000):
    f, psd_values = welch(signal, fs=fs, nperseg=1024, scaling='density')
    return psd_values

ECG_6_rearrange = np.reshape(ECG_6,(len(ECG_6)*7,3750))
ECG_7_rearrange = np.reshape(ECG_7,(len(ECG_7)*7,3750))
ECG_8_rearrange = np.reshape(ECG_8,(len(ECG_8)*7,3750))

entropy_6 = np.array([calculate_entropy(signal) for signal in ECG_6_rearrange])
entropy_6=entropy_6.reshape(len(entropy_6),1)
entropy_7 = np.array([calculate_entropy(signal) for signal in ECG_7_rearrange])
entropy_7=entropy_7.reshape(len(entropy_7),1)
entropy_8 = np.array([calculate_entropy(signal) for signal in ECG_8_rearrange])
entropy_8=entropy_8.reshape(len(entropy_8),1)

psd_6 = np.array([np.mean(calculate_psd(signal)) for signal in ECG_6_rearrange])
psd_6=psd_6.reshape(len(psd_6),1)
psd_7 = np.array([np.mean(calculate_psd(signal)) for signal in ECG_7_rearrange])
psd_7=psd_7.reshape(len(psd_7),1)
psd_8 = np.array([np.mean(calculate_psd(signal)) for signal in ECG_8_rearrange])
psd_8=psd_8.reshape(len(psd_8),1)

skew_6 = np.array([skew(signal) for signal in ECG_6_rearrange])
skew_6=skew_6.reshape(len(skew_6),1)
skew_7 = np.array([skew(signal) for signal in ECG_7_rearrange])
skew_7=skew_7.reshape(len(skew_7),1)
skew_8 = np.array([skew(signal) for signal in ECG_8_rearrange])
skew_8=skew_8.reshape(len(skew_8),1)

kurtosis_6 = np.array([kurtosis(signal) for signal in ECG_6_rearrange])
kurtosis_6=kurtosis_6.reshape(len(kurtosis_6),1)
kurtosis_7 = np.array([kurtosis(signal) for signal in ECG_7_rearrange])
kurtosis_7=kurtosis_7.reshape(len(kurtosis_7),1)
kurtosis_8 = np.array([kurtosis(signal) for signal in ECG_8_rearrange])
kurtosis_8=kurtosis_8.reshape(len(kurtosis_8),1)

std_6 = np.array([np.std(signal) for signal in ECG_6_rearrange])
std_6=std_6.reshape(len(std_6),1)
std_7 = np.array([np.std(signal) for signal in ECG_7_rearrange])
std_7=std_7.reshape(len(std_7),1)
std_8 = np.array([np.std(signal) for signal in ECG_8_rearrange])
std_8=std_8.reshape(len(std_8),1)

We_6 = np.array([np.sum(np.abs(pywt.wavedec(signal, wavelet='db3', level=1))) for signal in ECG_6_rearrange])
We_6=We_6.reshape(len(We_6),1)
We_7 = np.array([np.sum(np.abs(pywt.wavedec(signal, wavelet='db3', level=1))) for signal in ECG_7_rearrange])
We_7=We_7.reshape(len(We_7),1)
We_8 = np.array([np.sum(np.abs(pywt.wavedec(signal, wavelet='db3', level=1))) for signal in ECG_8_rearrange])
We_8=We_8.reshape(len(We_8),1)

Wn_6 = np.array([np.mean(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_6_rearrange])
Wn_6=Wn_6.reshape(len(Wn_6),1)
Wn_7 = np.array([np.mean(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_7_rearrange])
Wn_7=Wn_7.reshape(len(Wn_7),1)
Wn_8 = np.array([np.mean(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_8_rearrange])
Wn_8=Wn_8.reshape(len(Wn_8),1)

Ws_6 = np.array([np.std(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_6_rearrange])
Ws_6=Ws_6.reshape(len(Ws_6),1)
Ws_7 = np.array([np.std(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_7_rearrange])
Ws_7=Ws_7.reshape(len(Ws_7),1)
Ws_8 = np.array([np.std(pywt.wavedec(signal, wavelet='db3', level=1)) for signal in ECG_8_rearrange])
Ws_8=Ws_8.reshape(len(Ws_8),1)


max_6 = np.array([np.max(signal) for signal in ECG_6_rearrange])
max_6=max_6.reshape(len(max_6),1)
max_7 = np.array([np.max(signal) for signal in ECG_7_rearrange])
max_7=max_7.reshape(len(max_7),1)
max_8 = np.array([np.max(signal) for signal in ECG_8_rearrange])
max_8=max_8.reshape(len(max_8),1)

min_6 = np.array([np.min(signal) for signal in ECG_6_rearrange])
min_6=min_6.reshape(len(min_6),1)
min_7 = np.array([np.min(signal) for signal in ECG_7_rearrange])
min_7=min_7.reshape(len(min_7),1)
min_8 = np.array([np.min(signal) for signal in ECG_8_rearrange])
min_8=min_8.reshape(len(min_8),1)

mean_6 = np.array([np.mean(signal) for signal in pan_envelope_E_6])
mean_6=mean_6.reshape(len(mean_6),1)
mean_7 = np.array([np.mean(signal) for signal in pan_envelope_E_7])
mean_7=mean_7.reshape(len(mean_7),1)
mean_8 = np.array([np.mean(signal) for signal in pan_envelope_E_8])
mean_8=mean_8.reshape(len(mean_8),1)

Feature_6 = np.concatenate([dtw_dist_lag_6.reshape(len(dtw_dist_lag_6),1), correlations_lag_6.reshape(len(correlations_lag_6),1),entropy_6,psd_6,skew_6,kurtosis_6,std_6,We_6,Wn_6,Ws_6,max_6,min_6,mean_6,Feature_indices_6], axis=1)
Feature_7 = np.concatenate([dtw_dist_lag_7.reshape(len(dtw_dist_lag_7),1), correlations_lag_7.reshape(len(correlations_lag_7),1),entropy_7,psd_7,skew_7,kurtosis_7,std_7,We_7,Wn_7,Ws_7,max_7,min_7,mean_7,Feature_indices_7], axis=1)
Feature_8 = np.concatenate([dtw_dist_lag_8.reshape(len(dtw_dist_lag_8),1), correlations_lag_8.reshape(len(correlations_lag_8),1),entropy_8,psd_8,skew_8,kurtosis_8,std_8,We_8,Wn_8,Ws_8,max_8,min_8,mean_8,Feature_indices_8], axis=1)

Feature_label_all = np.concatenate([Feature_6,Feature_7,Feature_8]) 

In [ ]:
# Load the model
loaded_model = CatBoostClassifier()
loaded_model.load_model('/Users/alirezarafiei/Documents/DopplerGAN/catboost_classifier.cbm')
predictions = []
Features = [Feature_6,Feature_7,Feature_8]
for f in Features:
    # Make predictions
    preds = loaded_model.predict(f)
    predictions.append(preds.T[0].reshape(7,int(len(preds.T[0])/7)))

In [ ]:
subjects = ['BA120381V2','KF221288V2','SZ290877V2']
for j in range(len(subjects)):
    for i in range(1,8):
        np.savetxt(f'/Users/alirezarafiei/Documents/DopplerGAN/{subjects[j]}_{i}_SQI.txt', predictions[j][i-1], delimiter=',')